In [2]:
from keras.layers import Dense, Dropout, Embedding, Flatten, Input, merge
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.models import Model

from time import time
import datetime
from itertools import combinations
import pickle

import numpy as np
import pandas as pd
from scipy import sparse, mod
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [3]:
train = pd.read_csv('./data/train.csv')
train_label = train['target']
train_id = train['id']

del train['target'], train['id']

test = pd.read_csv('./data/test.csv')
test_id = test['id']
del test['id']

# 기초 통계 변수 생성

In [8]:
def proj_num_on_cat(train_df, test_df, target_column, group_column) :
    
    train_df['row_id'] = range(train_df.shape[0])
    test_df['row_id'] = range(test_df.shape[0])
    train_df['train'] = 1; test_df['train'] = 0
    
    #train + test
    all_df = train_df[['row_id','train',target_column, group_column]].append(test_df[['row_id','train',target_column, group_column]])
    grouped = all_df[[target_column, group_column]].groupby(group_column)
    
    #size, mean, median, max, min
    the_size = pd.DataFrame(grouped.size()).reset_index()
    the_size.columns = [group_column, '$s_size' % target_column]
    the_mean = pd.DataFrame(grouped.mean()).reset_index()
    the_mean.columns = [group_column, '$s_mean' % target_column]
    the_std = pd.DataFrame(grouped.std()).reset_index()
    the_std.columns = [group_column, '$s_std' % target_column]
    the_median = pd.DataFrame(grouped.median()).reset_index()
    the_median.columns = [group_column, '$s_median' % target_column]
    the_max = pd.DataFrame(grouped.max()).reset_index()
    the_max.columns = [group_column, '$s_max' % target_column]
    the_min = pd.DataFrame(grouped.min()).reset_index()
    the_min.columns = [group_column, '$s_min' % target_column]
    
    #통게 기반 파생 변수
    the_stats = pd.merge(the_size, the_mean)
    the_stats = pd.merge(the_stats, the_std)
    the_stats = pd.merge(the_stats, the_median)
    the_stats = pd.merge(the_stats, the_max)
    the_stats = pd.merge(the_stats, the_min)
    
    all_df = pd.merge(all_df, the_stats, how = 'left')
    
    #split train ,test
    selected_train = all_df[all_df.train == 1]
    selected_test = all_df[all_df.train == 0]
    
    selected_train.sort_values('row_id', inplace = True)
    selected_test.sort_values('row_id', inplace = True)
    
    selected_train.drop([target_column, group_column,'row_id','train'], axis = 1, inplace = True)
    selected_test.drop([target_column, group_column,'row_id','train'], axis = 1, inplace = True)
    
    selected_train, selected_test = np.array(selected_train), np.array(selected_test)
    
    return selected_train, selected_test